In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import path
import pandas as pd
import sys
import json
import torch
from torch.utils.data import DataLoader
import lightning.pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
import os
import wandb
from argparse import Namespace
import datetime

import util as tu
import model as tm
from setup import *


c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\TanJI\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
c:\Users\TanJI\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
config = Namespace (
    project_name = "TanJI",
    test_database_ids = [13],
    random_seed = 114514,
    batch_size = 1000,
    accelerator = 'cpu',
    progress_bar = True,
    lr = 0.001,
    dropout = 0.2,
    epochs = 200,
    pad_length = 20,
    node_length = 18,
    hidden_dim = 192,
    output_dim = 1,
    newpath = './results/full/cost/',
    to_predict = 'cost',
    dataset_file_name = 'alldataset.pkl',
)

In [12]:
# with open(path.join(ROOT_DIR, 'data', 'workload1', 'statistics.json')) as file:
#     feature_statistics = json.load(file)
with open(path.join(ROOT_DIR, 'data', 'imdb', 'statistics.json')) as file:
    feature_statistics = json.load(file)
encoder = tu.Encoder(feature_statistics)

{'Nested Loop': 0, 'Merge Join': 1, 'BitmapAnd': 2, 'Sort': 3, 'Hash Join': 4, 'Bitmap Index Scan': 5, 'Seq Scan': 6, 'Bitmap Heap Scan': 7, 'Materialize': 8, 'Hash': 9, 'Index Scan': 10, 'Gather Merge': 11, 'Gather': 12}
{'Actual Total Time': RobustScaler(), 'Plan Rows': RobustScaler(), 'Total Cost': RobustScaler()}


In [6]:
import pickle

# 存储类的实例
def save_instance(instance, filename):
    with open(filename, 'wb') as file:
        pickle.dump(instance, file)

# 加载存储的实例
def load_instance(filename):
    with open(filename, 'rb') as file:
        instance = pickle.load(file)
    return instance

In [13]:
# for zero-shot data
files_name = []

# for wk_item in workloads:
#     files_name.append(path.join(ROOT_DIR, 'data', 'workload1', wk_item + '_filted.json'))
for i in range(20):
    files_name.append(path.join(ROOT_DIR, 'data', 'imdb', 'plan_and_cost', 'train_plan_part{}.csv'.format(i)))

# data_file_name = path.join(ROOT_DIR, 'data', 'workload1', config.dataset_file_name)
data_file_name = path.join(ROOT_DIR, 'data', 'imdb', config.dataset_file_name)

if os.path.exists(data_file_name):
    alldataset = load_instance(data_file_name)
else:
    # dict_list = tu.Encoder.format_workload(files_name)
    dict_list = tu.Encoder.format_imdb(files_name)
    alldataset = tu.PlanTreeDataSet(dict_list, encoder)
    save_instance(alldataset, data_file_name)

torch.Size([100000, 1, 360]) torch.Size([100000, 20, 20]) torch.Size([100000, 1]) torch.Size([100000, 1])


In [14]:
train_dataloader, val_dataloader, test_dataloader = tu.get_dataloader(alldataset, config)

5 4 4


In [15]:
nowtime = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
model = tm.TanJI(config)
logger = pl.loggers.WandbLogger(project=config.project_name, name=nowtime)
logger.log_hyperparams(config.__dict__)
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath=os.path.join(ROOT_DIR, "checkpoints"),
    filename="DACE",
)
trainer = tm.PLTrainer(
    accelerator=config.accelerator,
    enable_progress_bar=config.progress_bar,
    enable_model_summary=config.progress_bar,
    max_epochs=config.epochs,
    logger=logger,
    callbacks=[checkpoint_callback],
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tanji. Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# trainer = PLTrainer(accelerator="cpu", max_epochs=50, logger=wandb_logger)
trainer.fit(model, train_dataloader, val_dataloader)

In [ ]:
# test model
result = trainer.test(model, dataloaders=test_dataloader)